# ccdt_afig_baselineHFA_byClus_lmmstats

### This notebook exports a CSV file that you can load into R for LMM stats

built from ccdt_afig_baselineHFA_byClus


Ashwin G. Ramayya  (ashwinramayya@gmail.com) 11/15/2023



In [1]:
#import packages
import numpy as np # numpy
import pandas as pd # pandas
import mne
import pickle
import os
from scipy.io import loadmat # to load matlab
from scipy import stats,ndimage,signal,spatial
import numpy.random as rand
import pycircstat as circ
import fooof as ff
import tensorpac as tp
import ccdt_func as cc
import pingouin as pg
#FDR correction
from statsmodels.stats.multitest import multipletests as FDR

# plotting
import matplotlib
#matplotlib.use('macosx')
#matplotlib.use('Qt5Agg')

# this makes tight axes 
matplotlib.rcParams['axes.autolimit_mode'] = 'round_numbers'
matplotlib.rcParams['axes.xmargin'] = 0
matplotlib.rcParams['axes.ymargin'] = 0
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['axes.spines.right'] = False
matplotlib.rcParams['axes.spines.top'] = False
matplotlib.rcParams['xtick.labelsize'] = 20
matplotlib.rcParams['ytick.labelsize'] = 20
matplotlib.rcParams['legend.fontsize'] = 20
matplotlib.rcParams['font.size'] = 20
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt 

# create fig dir
save_dir = os.getcwd()+'/stats/'
if os.path.exists(save_dir)==False:
     os.mkdir(save_dir)
        

/Users/ashwinramayya/Environments/ccdtenv/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


#### Here, we load processed data. load a Collection object, define parameters for power analysis, Load power data, We remove bad electrodes; 


Important Note To run 3d power:
- Default parameters  uses a wave_freq range of (70, 200) and 5 frequencies when computing power
- To compute power across other frequencies, we need to provide a new params dictionary
- This will create a separate folder for analyses (this is how each paramsdictionary will get tagged
- Then for subsequent analyses (like relating to brain regions or clusters, will need to develop methods to index these power data using independent boolans, rather than cluster electrode. May be easy to update plotPow3d to allow us to provide a boolean

In [2]:
%%capture 
paramsDict={}
paramsDict['wave_frange'] = (2,200)
paramsDict['wave_number_of_freq']=50
paramsDict['tmin_ms'] = -1000
paramsDict['tmax_ms'] =2500 #(+1500 gets you to color change, + 2500 gets you to the end of the response interval)


# get collection to filter subjCollection (for popStats)
Cpow = cc.Collection(collection_lbl='all',paramsDict=paramsDict)


# calculate 3d power for long delay trials ()
Cpow.getPow_3d(pow_evType='FIX_START',pow_evQuery = 'error==0&fastResponse==0&badTrial==0&delay==1500', do_zscore = True,apply_time_bins=True,time_bin_size_ms=50)


# filter to only include good electrodes (manually done above) and specific regions
goodElecsIdx = np.array(Cpow.isBadElectrode_list)==False

# option to only filter good electrodes
Cpow.filterElectrodes(filtE_bool= goodElecsIdx)



## Collecting data

Here we collect our standard set of processed data (that we use in rtClus, clusHFA,popHFA) that will allow us to index the power data by subject, anatomy, RT cluster, etc. Not we exclude bad electrodes from these data. Thus, there should be a one to one correspondence between power data and HFA data. 

In [3]:
%%capture 
# Collect Group taskstats Data
C = cc.Collection(collection_lbl='all')

C.doTaskStats_2d(pow_frange_lbl = 'HFA',
pow_method = 'wave',
pow_evQuery = 'error==0&fastResponse==0&badTrial==0',
do_zscore = True,
apply_gauss_smoothing = True,
gauss_sd_scaling = 0.075,
apply_time_bins=False,num_iters=1000,
time_bin_size_ms=100,overwriteFlag =False,feat_list_beh=['zrrtStoch'])

# get bool masks (for overall )
#ret_dict = C.groupElectrodesByTaskStats(print_flag=False)

# filter to only include good electrodes
C.filterElectrodes(filtE_bool= np.array(C.isBadElectrode_list)==False)


# Cluster Eelectrodes
feat_option = 'selectivity_taskRt'

C.clusterElectrodesByTaskStats(feat_option=feat_option,binarize_stats=True)

# get target-locked pow (long delay only)
C.getPow_2d(pow_evType='FIX_START',pow_frange_lbl = 'HFA',pow_method = 'wave',pow_evQuery = 'error==0&fastResponse==0&badTrial==0',paramsDict = {},do_zscore = True,apply_gauss_smoothing = True,gauss_sd_scaling = 0.075,apply_time_bins=True,time_bin_size_ms=50,overwriteFlag=False)

# create a shallow copy and get response-locked power
import copy
C2 = copy.copy(C)
C2.getPow_2d(pow_evType='RESPONSE',pow_frange_lbl = 'HFA',pow_method = 'wave',pow_evQuery = 'error==0&fastResponse==0&badTrial==0',paramsDict = {},do_zscore = True,apply_gauss_smoothing = True,gauss_sd_scaling = 0.075,apply_time_bins=True,time_bin_size_ms=50,overwriteFlag=False)

# get errors
C_err = copy.copy(C)
C_err.getPow_2d(pow_evType='FIX_START',pow_frange_lbl = 'HFA',pow_method = 'wave',pow_evQuery = 'RT<0&delay==1500',paramsDict = {},do_zscore = True,apply_gauss_smoothing = True,gauss_sd_scaling = 0.075,apply_time_bins=True,time_bin_size_ms=50,overwriteFlag=False)

# get errors
C2_err = copy.copy(C)
C2_err.getPow_2d(pow_evType='RESPONSE',pow_frange_lbl = 'HFA',pow_method = 'wave',pow_evQuery = 'RT<0&delay==1500',paramsDict = {},do_zscore = True,apply_gauss_smoothing = True,gauss_sd_scaling = 0.075,apply_time_bins=True,time_bin_size_ms=50,overwriteFlag=False)


# get ret_dict (to identify subgroups of electrodes)
ret_dict = C.groupElectrodesByTaskStats()

# get anatDf
anatDf,roiList = C.getAnatDf(ret_idx=None,cmap='rainbow',atlas='default');
anatDf_yeo,roiList_yeo = C.getAnatDf(ret_idx=None,cmap='rainbow',atlas='yeo');


# get behavioral data (we will add columns to this dataframe throughout this notebook)
beh_df = C.getBehXSubj(evQuery='error==0&fastResponse==0', later_model_type = 'distance',\
                      later_model_strategy = 'DynamicBaseline')


def get_clus_cut_level(num_levels_to_test = 30,min_subj = 5, min_elec=50):
    def get_num_generalized_clusters(cut_level,min_subj = 5, min_elec=50):
        #This function will compute how many clusters meet the miminum subject and electrode criteria for a given cluster level    clusid=-1

        clusid = -1
        num_clus = 0
        for i in np.arange(-1,cut_level):
            clusid+=1

            # ret_idx
            ret_idx = C.clus_cut_tree[:,cut_level]==clusid

            # calc num subj
            n_subj = np.count_nonzero(C.collapseBySubj_1d(ret_idx))

            if (n_subj>=min_subj) & (np.count_nonzero(ret_idx)>=min_elec):
                num_clus+=1
        return num_clus

    # plot number of "reproducible" clusters as a function of cluster level
    cut_levels_to_test = np.arange(0,num_levels_to_test,1)
    num_clus_by_level = np.zeros(len(cut_levels_to_test))
    count=-1
    for cut_level in cut_levels_to_test:
        count+=1
        num_clus_by_level[count] = get_num_generalized_clusters(cut_level = cut_level,min_subj = min_subj, min_elec=min_elec)
        #print(cut_level)

    cut_level = cut_levels_to_test[np.argmax(num_clus_by_level)]
    print('cut level = ',cut_level) 

    return cut_level, cut_levels_to_test,num_clus_by_level

# [] plot time course by subj (same plot) (fill in between) 
# [] plot time course of correlation to find peak value

# get number of clusters
cut_level,cut_levels_to_test,num_clus_by_level = \
get_clus_cut_level(num_levels_to_test = 10,min_subj = 20, min_elec=200) 

In [4]:
eIdx = 4
C.clus_cut_tree[eIdx,cut_level].astype('str')

'0'

In [25]:
# Make a Data frame for the full dataset

# time and freq of interest
frange = (70,200)
trange = (-250,1450)
trange_bl = (-500,0)

# list container for dict
eDat_list = []

#time range

for e in C.uElbl_list:
    
    # find electrode idx
    eIdx = np.where(C.uElbl_list==e)[0][0]
    f_idx = (Cpow.pow3d_freqs>=frange[0])&(Cpow.pow3d_freqs<=frange[1])
    t_idx = (Cpow.pow3d_xval>=trange[0])&(Cpow.pow3d_xval<=trange[1])
    t_idx_bl = (Cpow.pow3d_xval>=trange_bl[0])&(Cpow.pow3d_xval<=trange_bl[1])    
    
    
    #get pow values
    mean_pow = np.nanmean(np.nanmean(Cpow.pow3d[f_idx,:,eIdx],axis=0)[t_idx])
    mean_pow_bl = np.nanmean(np.nanmean(Cpow.pow3d[f_idx,:,eIdx],axis=0)[t_idx_bl])
    
    
    #make a dictionary to hold data for this electrode, time window
    eDat_dict = {}
    eDat_dict['pow'] = mean_pow
    if C.clus_cut_tree[eIdx,cut_level] == 0:
        eDat_dict['clusid'] = 'zero'# as string so it is categorical
    elif C.clus_cut_tree[eIdx,cut_level] == 1:
        eDat_dict['clusid'] = 'one'
    elif C.clus_cut_tree[eIdx,cut_level] == 2:
        eDat_dict['clusid'] = 'two'       
    elif C.clus_cut_tree[eIdx,cut_level] == 3:
        eDat_dict['clusid'] = 'three'
    else:
        continue # skip these electrodes
        
        
    eDat_dict['subj'] = C.subj_list[eIdx]
    eDat_dict['uElbl']=e
    eDat_dict['rtDiff_mean'] = beh_df.loc[C.subj_list[eIdx]]['rtDiff_mean']
    eDat_dict['error_diff'] = beh_df.loc[C.subj_list[eIdx]]['error_diff']
    eDat_dict['paramsDiff_P'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_P']
    eDat_dict['paramsDiff_D'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_D']
    eDat_dict['roiYeo'] = anatDf_yeo.iloc[eIdx]['roi']
    eDat_dict['roiRegion'] = anatDf.iloc[eIdx]['roi']
    #append the list 
    eDat_list.append(eDat_dict)
        
    
#make a dataframe
eDat_df=pd.DataFrame(eDat_list)

#save to csv
eDat_df.to_csv(save_dir+'baselineHFAByClus.csv')

In [21]:
np.isin(C.clus_cut_tree[30,cut_level],[1,3])==False
print(C.clus_cut_tree[30,cut_level])


1


In [24]:
# Make a Data frame for Clus 1 vs. 3 
# time and freq of interest
frange = (70,200)
trange1 = (-120,130)
trange2 = (1180,1430)

# list container for dict
eDat_list = []

#time range

for e in C.uElbl_list:
    
    # find electrode idx
    eIdx = np.where(C.uElbl_list==e)[0][0]
    
    #skip if we are not dealing with cluster 1 or cluster 3
    if np.isin(C.clus_cut_tree[eIdx,cut_level],[1,3])==False:
        continue
    
    
    f_idx = (Cpow.pow3d_freqs>=frange[0])&(Cpow.pow3d_freqs<=frange[1])
    t_idx1 = (Cpow.pow3d_xval>=trange1[0])&(Cpow.pow3d_xval<=trange1[1])
    t_idx2 = (Cpow.pow3d_xval>=trange2[0])&(Cpow.pow3d_xval<=trange2[1])    
    
    # fill in trange 1:
    
    #get pow values
    mean_pow = np.nanmean(np.nanmean(Cpow.pow3d[f_idx,:,eIdx],axis=0)[t_idx1])    
    
    #make a dictionary to hold data for this electrode, time window
    eDat_dict = {}
    eDat_dict['pow'] = mean_pow
    eDat_dict['time'] = 'early'
    
    #eDat_dict['pow_bl'] = mean_pow_bl
    if C.clus_cut_tree[eIdx,cut_level] == 0:
        eDat_dict['clusid'] = 'zero'# as string so it is categorical
    elif C.clus_cut_tree[eIdx,cut_level] == 1:
        eDat_dict['clusid'] = 'one'
    elif C.clus_cut_tree[eIdx,cut_level] == 2:
        eDat_dict['clusid'] = 'two'       
    elif C.clus_cut_tree[eIdx,cut_level] == 3:
        eDat_dict['clusid'] = 'three'
    else:
        eDat_dict['clusid'] = 'zero' # count it as the null cluster
   
        
    eDat_dict['subj'] = C.subj_list[eIdx]
    eDat_dict['uElbl']=e
    eDat_dict['rtDiff_mean'] = beh_df.loc[C.subj_list[eIdx]]['rtDiff_mean']
    eDat_dict['error_diff'] = beh_df.loc[C.subj_list[eIdx]]['error_diff']
    eDat_dict['paramsDiff_P'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_P']
    eDat_dict['paramsDiff_D'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_D']
    eDat_dict['roiYeo'] = anatDf_yeo.iloc[eIdx]['roi']
    eDat_dict['roiRegion'] = anatDf.iloc[eIdx]['roi']
    #append the list 
    eDat_list.append(eDat_dict)
        
        
    # fill in time range 2
    #get pow values
    mean_pow = np.nanmean(np.nanmean(Cpow.pow3d[f_idx,:,eIdx],axis=0)[t_idx2])    
    
    #make a dictionary to hold data for this electrode, time window
    eDat_dict = {}
    eDat_dict['pow'] = mean_pow
    eDat_dict['time'] = 'late'
    
    if C.clus_cut_tree[eIdx,cut_level] == 0:
        eDat_dict['clusid'] = 'zero'# as string so it is categorical
    elif C.clus_cut_tree[eIdx,cut_level] == 1:
        eDat_dict['clusid'] = 'one'
    elif C.clus_cut_tree[eIdx,cut_level] == 2:
        eDat_dict['clusid'] = 'two'       
    elif C.clus_cut_tree[eIdx,cut_level] == 3:
        eDat_dict['clusid'] = 'three'
    else:
        eDat_dict['clusid'] = 'zero' # count it as the null cluster
   
        
    eDat_dict['subj'] = C.subj_list[eIdx]
    eDat_dict['uElbl']=e
    eDat_dict['rtDiff_mean'] = beh_df.loc[C.subj_list[eIdx]]['rtDiff_mean']
    eDat_dict['error_diff'] = beh_df.loc[C.subj_list[eIdx]]['error_diff']
    eDat_dict['paramsDiff_P'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_P']
    eDat_dict['paramsDiff_D'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_D']
    eDat_dict['roiYeo'] = anatDf_yeo.iloc[eIdx]['roi']
    eDat_dict['roiRegion'] = anatDf.iloc[eIdx]['roi']
    
    #append the list 
    eDat_list.append(eDat_dict)    
#make a dataframe
eDat_df=pd.DataFrame(eDat_list)

#save to csv
eDat_df.to_csv(save_dir+'CLUS1V3.csv')

In [10]:
# Make a Data frame for cluster 1 only

# time and freq of interest
frange = (70,200)
trange = (1200,1450)
trange_bl = (-500,0)

# list container for dict
eDat_list = []

#time range

for e in C.uElbl_list:
    
    # find electrode idx
    eIdx = np.where(C.uElbl_list==e)[0][0]
    
    #skip if we are not dealing with cluster 1
    if (C.clus_cut_tree[eIdx,cut_level]==1)==False:
        continue
    
    
    f_idx = (Cpow.pow3d_freqs>=frange[0])&(Cpow.pow3d_freqs<=frange[1])
    t_idx = (Cpow.pow3d_xval>=trange[0])&(Cpow.pow3d_xval<=trange[1])
    t_idx_bl = (Cpow.pow3d_xval>=trange_bl[0])&(Cpow.pow3d_xval<=trange_bl[1])    
    
    
    #get pow values
    mean_pow = np.nanmean(np.nanmean(Cpow.pow3d[f_idx,:,eIdx],axis=0)[t_idx])
    #mean_pow_bl = np.nanmean(np.nanmean(Cpow.pow3d[f_idx,:,eIdx],axis=0)[t_idx_bl])
    
    
    #make a dictionary to hold data for this electrode, time window
    eDat_dict = {}
    eDat_dict['pow'] = mean_pow
    #eDat_dict['pow_bl'] = mean_pow_bl
    if C.clus_cut_tree[eIdx,cut_level] == 0:
        eDat_dict['clusid'] = 'zero'# as string so it is categorical
    elif C.clus_cut_tree[eIdx,cut_level] == 1:
        eDat_dict['clusid'] = 'one'
    elif C.clus_cut_tree[eIdx,cut_level] == 2:
        eDat_dict['clusid'] = 'two'       
    elif C.clus_cut_tree[eIdx,cut_level] == 3:
        eDat_dict['clusid'] = 'three'
    else:
        eDat_dict['clusid'] = 'zero' # count it as the null cluster
        
        
        
    eDat_dict['subj'] = C.subj_list[eIdx]
    eDat_dict['uElbl']=e
    eDat_dict['rtDiff_mean'] = beh_df.loc[C.subj_list[eIdx]]['rtDiff_mean']
    eDat_dict['error_diff'] = beh_df.loc[C.subj_list[eIdx]]['error_diff']
    eDat_dict['paramsDiff_P'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_P']
    eDat_dict['paramsDiff_D'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_D']
    eDat_dict['roiYeo'] = anatDf_yeo.iloc[eIdx]['roi']
    eDat_dict['roiRegion'] = anatDf.iloc[eIdx]['roi']
    #append the list 
    eDat_list.append(eDat_dict)
        
    
#make a dataframe
eDat_df=pd.DataFrame(eDat_list)

#save to csv
eDat_df.to_csv(save_dir+'baselineHFAByClus-clus1.csv')

In [8]:
# Make a Data frame for cluster 3 only

# time and freq of interest
frange = (70,200)
trange = (-100,150)
trange_bl = (-500,0)

# list container for dict
eDat_list = []

#time range

for e in C.uElbl_list:
    
    # find electrode idx
    eIdx = np.where(C.uElbl_list==e)[0][0]
    
    #skip if we are not dealing with cluster 1
    if (C.clus_cut_tree[eIdx,cut_level]==3)==False:
        continue
    
    
    f_idx = (Cpow.pow3d_freqs>=frange[0])&(Cpow.pow3d_freqs<=frange[1])
    t_idx = (Cpow.pow3d_xval>=trange[0])&(Cpow.pow3d_xval<=trange[1])
    t_idx_bl = (Cpow.pow3d_xval>=trange_bl[0])&(Cpow.pow3d_xval<=trange_bl[1])    
    
    
    #get pow values
    mean_pow = np.nanmean(np.nanmean(Cpow.pow3d[f_idx,:,eIdx],axis=0)[t_idx])
    
    
    #make a dictionary to hold data for this electrode, time window
    eDat_dict = {}
    eDat_dict['pow'] = mean_pow
    #eDat_dict['pow_bl'] = mean_pow_bl
    if C.clus_cut_tree[eIdx,cut_level] == 0:
        eDat_dict['clusid'] = 'zero'# as string so it is categorical
    elif C.clus_cut_tree[eIdx,cut_level] == 1:
        eDat_dict['clusid'] = 'one'
    elif C.clus_cut_tree[eIdx,cut_level] == 2:
        eDat_dict['clusid'] = 'two'       
    elif C.clus_cut_tree[eIdx,cut_level] == 3:
        eDat_dict['clusid'] = 'three'
    else:
        eDat_dict['clusid'] = 'zero' # count it as the null cluster
        
        
        
    eDat_dict['subj'] = C.subj_list[eIdx]
    eDat_dict['uElbl']=e
    eDat_dict['rtDiff_mean'] = beh_df.loc[C.subj_list[eIdx]]['rtDiff_mean']
    eDat_dict['error_diff'] = beh_df.loc[C.subj_list[eIdx]]['error_diff']
    eDat_dict['paramsDiff_P'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_P']
    eDat_dict['paramsDiff_D'] = beh_df.loc[C.subj_list[eIdx]]['paramsDiff_D']
    eDat_dict['roiYeo'] = anatDf_yeo.iloc[eIdx]['roi']
    eDat_dict['roiRegion'] = anatDf.iloc[eIdx]['roi']
    #append the list 
    eDat_list.append(eDat_dict)
        
    
#make a dataframe
eDat_df=pd.DataFrame(eDat_list)

#save to csv
eDat_df.to_csv(save_dir+'baselineHFAByClus-clus3.csv')